# 통합 데이터베이스 관리 시스템

이 노트북은 다음 두 가지 기능을 통합합니다:
1. **성적처리 시스템**: 학생 성적 관리 (추가, 수정, 삭제, 조회)
2. **주급계산 시스템**: 직원 주급 계산 (근무시간, 시급, 초과근무 수당)

## 사용 기술
- **데이터베이스**: MySQL
- **라이브러리**: PyMySQL
- **기능**: CRUD 작업, 데이터 계산 및 분석


In [ ]:
# 라이브러리 import 및 데이터베이스 연결 설정
import pymysql
from pymysql import cursors
from datetime import datetime

# MySQL 데이터베이스 연결
conn = pymysql.connect(
    host='localhost',
    user='user01',
    password='1234',
    db='mydb',
    port=3306
)

# 커서 설정 (딕셔너리 형태로 결과 반환)
curs = conn.cursor(pymysql.cursors.DictCursor)

print("✅ 데이터베이스 연결 성공!")


# 📊 성적처리 시스템

학생들의 성적을 관리하는 시스템입니다.

## 주요 기능
- **목록 조회**: 모든 학생의 성적 조회
- **성적 추가**: 새 학생 성적 입력
- **성적 수정**: 기존 학생 성적 수정
- **성적 삭제**: 학생 성적 삭제
- **자동 계산**: 총점 및 평균 자동 계산


In [ ]:
# 성적처리 시스템 함수들

def score_insert():
    """새 학생 성적 추가"""
    print("\n=== 새 학생 성적 추가 ===")
    name = input("이름: ")
    kor = input("국어: ")
    eng = input("영어: ")
    mat = input("수학: ")
    
    sql = """
        INSERT INTO tb_score(sname, kor, eng, mat, regdate)
        VALUES (%s, %s, %s, %s, NOW())
    """
    curs.execute(sql, (name, kor, eng, mat))
    conn.commit()
    print("✅ 학생 성적이 추가되었습니다!")

def score_update():
    """기존 학생 성적 수정"""
    print("\n=== 학생 성적 수정 ===")
    id = input("수정할 학생 ID: ")
    name = input("이름: ")
    kor = input("국어: ")
    eng = input("영어: ")
    mat = input("수학: ")
    
    sql = """
        UPDATE tb_score
        SET sname = %s, kor = %s, eng = %s, mat = %s
        WHERE id = %s
    """
    curs.execute(sql, (name, kor, eng, mat, id))
    conn.commit()
    print("✅ 학생 성적이 수정되었습니다!")

def score_delete():
    """학생 성적 삭제"""
    print("\n=== 학생 성적 삭제 ===")
    id = input("삭제할 학생 ID: ")
    
    sql = "DELETE FROM tb_score WHERE id = %s"
    curs.execute(sql, (id,))
    conn.commit()
    print("🗑️ 학생 성적이 삭제되었습니다!")

def score_output():
    """모든 학생 성적 조회"""
    print("\n=== 📊 전체 학생 성적 목록 ===")
    sql = """
        SELECT id, sname, kor, eng, mat,
               (kor + eng + mat) AS total,
               ROUND((kor + eng + mat) / 3, 2) AS average
        FROM tb_score
        ORDER BY id
    """
    curs.execute(sql)
    rows = curs.fetchall()
    
    if not rows:
        print("❌ 등록된 학생이 없습니다.")
        return
    
    print(f"{'ID':<4} {'이름':<8} {'국어':<4} {'영어':<4} {'수학':<4} {'총점':<4} {'평균':<6}")
    print("-" * 45)
    
    for row in rows:
        print(f"{row['id']:<4} {row['sname']:<8} {row['kor']:<4} {row['eng']:<4} {row['mat']:<4} {row['total']:<4} {row['average']:<6}")

def score_menu():
    """성적처리 메뉴"""
    while True:
        print("\n" + "="*40)
        print("📊 성적처리 시스템")
        print("="*40)
        print("1. 성적 목록 조회")
        print("2. 성적 추가")
        print("3. 성적 수정")
        print("4. 성적 삭제")
        print("0. 메인 메뉴로 돌아가기")
        print("="*40)
        
        choice = input("선택하세요: ")
        
        if choice == "1":
            score_output()
        elif choice == "2":
            score_insert()
        elif choice == "3":
            score_update()
        elif choice == "4":
            score_delete()
        elif choice == "0":
            break
        else:
            print("❌ 올바른 번호를 입력해주세요.")

print("✅ 성적처리 시스템 함수들이 정의되었습니다!")


# 💰 주급계산 시스템

직원들의 주급을 계산하는 시스템입니다.

## 주요 기능
- **테이블 생성**: 주급 데이터를 저장할 테이블 자동 생성
- **직원 정보 추가**: 이름, 근무시간, 시급 입력
- **주급 계산**: 기본급 + 초과근무 수당 계산
- **초과근무 수당**: 20시간 초과 시 1.5배 가산

## 계산 규칙
- **기본 주급**: 근무시간 × 시급
- **초과근무**: 20시간 초과 시 초과분에 대해 1.5배 지급


In [ ]:
# 주급계산 시스템 함수들

def create_weekly_pay_table():
    """주급 테이블 생성"""
    sql = """
        CREATE TABLE IF NOT EXISTS weekly_pay (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(50) NOT NULL,
            work_hours INT NOT NULL,
            hourly_wage DECIMAL(10, 2) NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """
    curs.execute(sql)
    conn.commit()
    print("✅ weekly_pay 테이블이 준비되었습니다!")

def add_employee():
    """직원 정보 추가"""
    print("\n=== 직원 정보 추가 ===")
    name = input("직원 이름: ")
    work_hours = int(input("근무시간 (시간): "))
    hourly_wage = float(input("시급 (원): "))
    
    sql = """
        INSERT INTO weekly_pay (name, work_hours, hourly_wage)
        VALUES (%s, %s, %s)
    """
    curs.execute(sql, (name, work_hours, hourly_wage))
    conn.commit()
    print("✅ 직원 정보가 추가되었습니다!")

def calculate_all_weekly_pay():
    """모든 직원의 주급 계산 및 조회"""
    print("\n=== 💰 주급 계산 결과 ===")
    sql = """
        SELECT 
            id, name, work_hours, hourly_wage,
            (work_hours * hourly_wage) AS basic_pay,
            CASE 
                WHEN work_hours > 20 THEN (work_hours - 20) * hourly_wage * 1.5
                ELSE 0 
            END AS overtime_pay,
            CASE 
                WHEN work_hours > 20 THEN 
                    (20 * hourly_wage) + ((work_hours - 20) * hourly_wage * 1.5)
                ELSE work_hours * hourly_wage
            END AS total_pay
        FROM weekly_pay
        ORDER BY id
    """
    curs.execute(sql)
    rows = curs.fetchall()
    
    if not rows:
        print("❌ 등록된 직원이 없습니다.")
        return
    
    print(f"{'ID':<3} {'이름':<8} {'근무시간':<6} {'시급':<8} {'기본급':<10} {'초과수당':<10} {'총급여':<10}")
    print("-" * 70)
    
    total_basic = 0
    total_overtime = 0
    total_all = 0
    
    for row in rows:
        basic_pay = row['basic_pay']
        overtime_pay = row['overtime_pay']
        total_pay = row['total_pay']
        
        total_basic += basic_pay
        total_overtime += overtime_pay
        total_all += total_pay
        
        print(f"{row['id']:<3} {row['name']:<8} {row['work_hours']:<6} {row['hourly_wage']:<8} {basic_pay:<10} {overtime_pay:<10} {total_pay:<10}")
    
    print("-" * 70)
    print(f"{'총계':<18} {'':>10} {total_basic:<10} {total_overtime:<10} {total_all:<10}")

def delete_employee():
    """직원 정보 삭제"""
    print("\n=== 직원 정보 삭제 ===")
    employee_id = input("삭제할 직원 ID: ")
    
    sql = "DELETE FROM weekly_pay WHERE id = %s"
    curs.execute(sql, (employee_id,))
    conn.commit()
    print("🗑️ 직원 정보가 삭제되었습니다!")

def show_employee_list():
    """직원 목록 조회"""
    print("\n=== 👥 직원 목록 ===")
    sql = "SELECT id, name, work_hours, hourly_wage FROM weekly_pay ORDER BY id"
    curs.execute(sql)
    rows = curs.fetchall()
    
    if not rows:
        print("❌ 등록된 직원이 없습니다.")
        return
    
    print(f"{'ID':<3} {'이름':<8} {'근무시간':<8} {'시급':<8}")
    print("-" * 30)
    
    for row in rows:
        print(f"{row['id']:<3} {row['name']:<8} {row['work_hours']:<8} {row['hourly_wage']:<8}")

def weekly_pay_menu():
    """주급계산 메뉴"""
    create_weekly_pay_table()  # 테이블 생성
    
    while True:
        print("\n" + "="*40)
        print("💰 주급계산 시스템")
        print("="*40)
        print("1. 직원 목록 조회")
        print("2. 직원 정보 추가")
        print("3. 주급 계산")
        print("4. 직원 정보 삭제")
        print("0. 메인 메뉴로 돌아가기")
        print("="*40)
        
        choice = input("선택하세요: ")
        
        if choice == "1":
            show_employee_list()
        elif choice == "2":
            add_employee()
        elif choice == "3":
            calculate_all_weekly_pay()
        elif choice == "4":
            delete_employee()
        elif choice == "0":
            break
        else:
            print("❌ 올바른 번호를 입력해주세요.")

print("✅ 주급계산 시스템 함수들이 정의되었습니다!")


# 🎯 통합 메인 메뉴 시스템

두 시스템을 통합하여 사용할 수 있는 메인 메뉴입니다.

## 사용 방법
1. 아래 셀을 실행하여 메인 메뉴를 시작합니다
2. 원하는 기능을 선택하여 사용합니다
3. 각 시스템에서 0을 입력하면 메인 메뉴로 돌아옵니다

## 주의사항
- 데이터베이스 연결이 필요합니다
- 해당 테이블이 존재하지 않으면 자동으로 생성됩니다


In [ ]:
# 통합 메인 메뉴 시스템

def main_menu():
    """메인 메뉴 시스템"""
    print("\n🎉 통합 데이터베이스 관리 시스템에 오신 것을 환영합니다!")
    
    while True:
        print("\n" + "="*50)
        print("🎯 통합 데이터베이스 관리 시스템")
        print("="*50)
        print("1. 📊 성적처리 시스템")
        print("2. 💰 주급계산 시스템")
        print("3. 🔌 데이터베이스 연결 상태 확인")
        print("0. 🚪 프로그램 종료")
        print("="*50)
        
        choice = input("원하는 기능을 선택하세요: ")
        
        if choice == "1":
            print("\n📊 성적처리 시스템으로 이동합니다...")
            score_menu()
        elif choice == "2":
            print("\n💰 주급계산 시스템으로 이동합니다...")
            weekly_pay_menu()
        elif choice == "3":
            check_database_connection()
        elif choice == "0":
            print("\n👋 프로그램을 종료합니다.")
            break
        else:
            print("❌ 올바른 번호를 입력해주세요.")

def check_database_connection():
    """데이터베이스 연결 상태 확인"""
    try:
        curs.execute("SELECT 1")
        result = curs.fetchone()
        if result:
            print("✅ 데이터베이스 연결 상태: 정상")
            
            # 테이블 존재 여부 확인
            curs.execute("SHOW TABLES")
            tables = curs.fetchall()
            
            print("\n📋 현재 데이터베이스의 테이블 목록:")
            if tables:
                for table in tables:
                    table_name = list(table.values())[0]
                    curs.execute(f"SELECT COUNT(*) as count FROM {table_name}")
                    count = curs.fetchone()
                    print(f"  • {table_name}: {count['count']}개 레코드")
            else:
                print("  • 테이블이 없습니다.")
                
    except Exception as e:
        print(f"❌ 데이터베이스 연결 오류: {e}")

def cleanup_connection():
    """데이터베이스 연결 정리"""
    if conn:
        conn.close()
        print("🔌 데이터베이스 연결이 종료되었습니다.")

print("✅ 통합 메인 메뉴 시스템이 준비되었습니다!")


In [ ]:
# 프로그램 실행
# 아래 주석을 해제하고 실행하세요

# main_menu()

# 프로그램 종료 시 연결 정리
# cleanup_connection()

print("🎯 통합 데이터베이스 관리 시스템이 준비되었습니다!")
print("📝 사용 방법:")
print("1. 위의 주석을 해제하고 main_menu() 를 실행하세요")
print("2. 메뉴에서 원하는 기능을 선택하세요")
print("3. 각 시스템에서 0을 입력하면 메인 메뉴로 돌아갑니다")
print("4. 프로그램 종료 시 cleanup_connection() 를 실행하세요")


# 📋 사용법 및 예시

## 🔧 설정 요구사항
1. **MySQL 서버 실행 중**
2. **데이터베이스 'mydb' 존재**
3. **사용자 계정 'user01' (비밀번호: '1234') 권한 설정**

## 🎯 기능별 사용법

### 📊 성적처리 시스템
- **목록 조회**: 등록된 모든 학생의 성적과 총점, 평균을 확인
- **성적 추가**: 새 학생의 국어, 영어, 수학 점수 입력
- **성적 수정**: 기존 학생의 점수 수정
- **성적 삭제**: 특정 학생의 성적 삭제

### 💰 주급계산 시스템
- **직원 목록**: 등록된 모든 직원 정보 조회
- **정보 추가**: 새 직원의 이름, 근무시간, 시급 입력
- **주급 계산**: 
  - 기본급: 근무시간 × 시급
  - 초과근무 수당: 20시간 초과분 × 시급 × 1.5
  - 총 급여: 기본급 + 초과근무 수당
- **정보 삭제**: 특정 직원 정보 삭제

## 🚨 주의사항
- 데이터베이스 연결 실패 시 연결 정보를 확인하세요
- 테이블이 존재하지 않으면 자동으로 생성됩니다
- 숫자 입력이 필요한 필드에는 반드시 숫자를 입력하세요

## 🔧 문제 해결
- **연결 오류**: MySQL 서버 실행 상태 및 계정 정보 확인
- **권한 오류**: 사용자 계정에 해당 데이터베이스 권한 부여
- **테이블 오류**: 프로그램이 자동으로 테이블을 생성하므로 CREATE 권한 필요
